In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/IDS_575')
!ls

 advance_final_analysis.ipynb	 RestaurantReviewDataYelp.csv
 basic_approach_analysis.ipynb	'yelp reviews.ipynb'
 lexicon_based_approach.ipynb


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
 yelp = pd.read_csv('RestaurantReviewDataYelp.csv')

In [0]:
yelp.shape

(47495, 26)

In [5]:
yelp.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'type', 'useful', 'user_id', 'address', 'attributes[0]',
       'attributes[1]', 'attributes[2]', 'categories[0]', 'categories[1]',
       'categories[2]', 'city', 'is_open', 'latitude', 'longitude', 'name',
       'neighborhood', 'postal_code', 'review_count', 'state'],
      dtype='object')

In [0]:
#creating fucntion for lebelling data set for training purpose.
def new_col(row):
    if row['stars'] == 1 or row['stars'] ==2:
        val = 'negative'
    elif row['stars']==4 or row['stars']==5:
        val = 'positive'
    else:
        val = 'neutral'
    return val

In [0]:
yelp['sentiment'] = yelp.apply(new_col, axis=1)

In [8]:
yelp.shape

(47495, 27)

In [10]:
yelp.sentiment.value_counts()

positive    30133
negative     9981
neutral      7381
Name: sentiment, dtype: int64

In [0]:
#simple cleaning function to remove punctuations and spaces.........
#converting to lower case......
#no other method of data cleaning employed......
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"*()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def process_text(document):
    document = [REPLACE_NO_SPACE.sub("", line.lower()) for line in document]
    document = [REPLACE_WITH_SPACE.sub(" ", line) for line in document]
    
    return document

In [0]:
yelp['clean_text'] = process_text(yelp.text)

In [13]:
yelp['clean_text'][500]

'not bad sushi quiet and relax atmospherenthe service is good the place is clean and the menu is reasonable in price'

Note: There are a lot of different and more sophisticated ways to clean text data that would likely produce better results than what we’ve done here. we wanted part 1 of this to be as simple as possible. Also, I generally think it’s best to get baseline predictions with the simplest possible solution before spending time doing potentially unnecessary transformations.

In order for this data to make sense to our machine learning algorithm we’ll need to convert each review to a numeric representation, which we call vectorization.
The simplest form of this is to create one very large matrix with one column for every unique word in your corpus (where the corpus is all 50k reviews in our case). Then we transform each review into one row containing 0s and 1s, where 1 means that the word in the corpus corresponding to that column appears in that review. That being said, each row of the matrix will be very sparse (mostly zeros). This process is also known as one hot encoding.

In [14]:
import nltk
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
X_yelp = yelp["clean_text"]
y_yelp = yelp["sentiment"]

In [16]:
y_yelp[2]

'positive'

In [37]:

from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(binary=True).fit(X_yelp)
print(len(bow_transformer.vocabulary_))

102626


In [18]:
text25 = X_yelp[25]
text25

'i ordered for pick up and got the daal and mountain potatoes both dishes were excellent really quite tasty im impressed what im not impressed with is the prices    its just hard for me to fathom such prices for a small amount of desi foodnni hear the lunch buffet is much better bang for your buck so ill try that'

In [19]:
text25 = bow_transformer.transform([text25])
text25

<1x102626 sparse matrix of type '<class 'numpy.int64'>'
	with 48 stored elements in Compressed Sparse Row format>

In [20]:
#if you want to check name of features.................
print(bow_transformer.get_feature_names()[88205])
print(bow_transformer.get_feature_names()[64623])

tastedthe
optionsnni


In [0]:
X_yelp_t = bow_transformer.transform(X_yelp)

In [0]:
#calculating shape of sparse matrix and number of zeroes.......
print('Shape of Sparse Matrix: ', X_yelp_t.shape)
print('Amount of Non-Zero occurences: ', X_yelp_t.nnz)

Shape of Sparse Matrix:  (47495, 102626)
Amount of Non-Zero occurences:  3187754


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_yelp_t,y_yelp,test_size =.10)

Building LOgistic regression classifier and checking important features...............to keep things simple we are only going to worry about the hyperparameter C, which adjusts the regularization.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [24]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.791578947368421
Accuracy for C=0.05: 0.7951578947368421
Accuracy for C=0.25: 0.796
Accuracy for C=0.5: 0.7917894736842105
Accuracy for C=1: 0.7903157894736842


In [0]:
#maximum accuracy comes at c=.05.......................

Building naive bayes classifer............

In [0]:

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [26]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
preds = nb.predict(X_test)

In [28]:
accuracy_score(y_test, nb.predict(X_test))

0.7682105263157895

In [40]:
#using SVM for same setting..............
from sklearn.svm import LinearSVC
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))

Accuracy for C=0.01: 0.8012631578947368
Accuracy for C=0.05: 0.7991578947368421
Accuracy for C=0.25: 0.783578947368421
Accuracy for C=0.5: 0.7730526315789473
Accuracy for C=1: 0.7650526315789473


In [30]:

from sklearn.feature_extraction.text import CountVectorizer
bow_transformer_2 = CountVectorizer(binary=False).fit(X_yelp)
print(len(bow_transformer_2.vocabulary_))

102626


In [0]:
X_yelp_2 = bow_transformer_2.transform(X_yelp)

In [32]:
#calculating shape of sparse matrix and number of zeroes.......
print('Shape of Sparse Matrix: ', X_yelp_2.shape)
print('Amount of Non-Zero occurences: ', X_yelp_2.nnz)

Shape of Sparse Matrix:  (47495, 102626)
Amount of Non-Zero occurences:  3187754


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_yelp_2,y_yelp,test_size =.10)

In [34]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr_2 = LogisticRegression(C=c)
    lr_2.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.8654736842105263
Accuracy for C=0.05: 0.8654736842105263
Accuracy for C=0.25: 0.8654736842105263
Accuracy for C=0.5: 0.8654736842105263
Accuracy for C=1: 0.8654736842105263


In [35]:
nb.fit(X_train, y_train)
accuracy_score(y_test, nb.predict(X_test))

0.7738947368421053

In [36]:
from sklearn.svm import LinearSVC
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))

Accuracy for C=0.01: 0.8058947368421052
Accuracy for C=0.05: 0.8012631578947368
Accuracy for C=0.25: 0.7905263157894736
Accuracy for C=0.5: 0.7825263157894737
Accuracy for C=1: 0.7749473684210526


We have observed that naive bayes predicts accuracy score of 76% in this data.